Create a streamlit dashboard to consume data from the mart schema. Design the dashboard in a way to
help talent acquisition specialists working for specific occupation field your group is working on. For
instance, try to produce visualizations to answer these questions:
- for a specific occupation field (i.e. Data/IT), which occupation (i.e. data engineer) has a higher number of vacanies?
- which cities has a higher number of vacancies?
- etc
______________
You should include at least four meaningful KPI/metrics and visualizations on your dashboard that are able to improve efficiency of the work of talent acquisition specialists in this HR agency.

In [ ]:
import pandas as pd
import duckdb

with duckdb.connect("ads_data_warehouse.duckdb") as connection:
    df_mart = connection.execute("SELECT * FROM mart.mart_ads").df()
    df_education = connection.execute("SELECT * FROM mart.mart_pedagogik").df()
    df_healthcare = connection.execute("SELECT * FROM mart.mart_sjukvård").df()
    df_technical = connection.execute("SELECT * FROM mart.mart_teknisk").df()
    df_total_vacancies = connection.execute(
        "SELECT * FROM mart.mart_total_vacancies"
    ).df()


# i varje df, vilket occupation har högst andel lediga tjänster
# i varje df,  vilken stad har högst andel lediga tjänster
df_mart["scope_of_work_min"].isna().sum()
df_mart["occupation_field"]
df_total_vacancies

,occupation,workplace_municipality,occupation_field,total_vacancies
0,Vårdbiträde,Göteborg,Hälso- och sjukvård,18180.0
1,Elevassistent,Malmö,Pedagogik,2400.0
2,"Grundlärare, 4-6",Göteborg,Pedagogik,1114.0
3,Elevassistent,Lund,Pedagogik,1105.0
4,Lärarassistent,Linköping,Pedagogik,1075.0
...,...,...,...,...
3527,Universitets- och högskoleadjunkt,Skövde,Pedagogik,1.0
3528,Universitets- och högskoleadjunkt,Halmstad,Pedagogik,1.0
3529,Studie- och yrkesvägledare,Örnsköldsvik,Pedagogik,1.0
3530,Ungdomsinstruktör/Ungdomstränare,Umeå,Pedagogik,1.0


In [49]:
# i df:arna, vilket occupation har högst andel lediga tjänster
def vacancies_by_occupation(df):
    return df.groupby("occupation")["vacancies"].sum().sort_values(ascending=False).reset_index(name="total")

technical_vacancies_by_occ = vacancies_by_occupation(df_technical)
education_vacancies_by_occ = vacancies_by_occupation(df_education)
healthcare_vacancies_by_occ = vacancies_by_occupation(df_healthcare)
healthcare_vacancies_by_occ

,occupation,total
0,Vårdbiträde,18714
1,"Undersköterska, hemtjänst, äldreboende och hab...",5313
2,"Sjuksköterska, grundutbildad",3938
3,"Undersköterska, vård- och specialavdelning och...",851
4,Specialistläkare,650
...,...,...
66,Laboratoriebiträde,1
67,"Laboratoriebiträde, sjukhus",1
68,Individualterapeut,1
69,Hemvårdare,1


In [50]:
# i varje df,  vilken stad har högst andel lediga tjänster
def vacancies_by_city(df):
    return df.groupby("workplace_municipality")["vacancies"].sum().sort_values(ascending=False).reset_index()

technical_vacancies_by_city = vacancies_by_city(df_technical)
education_vacancies_by_city = vacancies_by_city(df_education)
healthcare_vacancies_by_city = vacancies_by_city(df_healthcare)
healthcare_vacancies_by_city
education_vacancies_by_city
technical_vacancies_by_city.head(3)

,workplace_municipality,vacancies
0,Karlstad,1586
1,Linköping,1218
2,Stockholm,1158


In [51]:
def vacancies(df, input):
    """INDATA: df, input att groupa med
    vacancies: sum, sort, reset
    """
    return (
        df.groupby(input)["vacancies"].sum().sort_values(ascending=False).reset_index(name=f"total vacancies")
    )

In [52]:
city = "workplace_municipality"
occupation = "occupation"
df_test = vacancies(df_technical, city)
df_test

,workplace_municipality,total vacancies
0,Karlstad,1586
1,Linköping,1218
2,Stockholm,1158
3,Göteborg,608
4,Södertälje,471
...,...,...
186,Vänersborg,1
187,Vaggeryd,1
188,Älvdalen,1
189,Öckerö,1


## uträkningar till dashboard

In [53]:
# 1. totalt antal lediga jobb. # totalt antal städer. 
total_citys = df_mart["workplace_municipality"].nunique()
total_vacancies = df_total_vacancies["total_vacancies"].sum()

total_vacancies


np.float64(70340.0)

In [54]:
# 2.TOP 3: yrket med mest lediga jobb. TOP 3 regioner med mest lediga jobb 
# 3. något mer? 

df_mart.groupby("workplace_region")["vacancies"].sum().sort_values(ascending=False).reset_index()
top_occupations = df_mart.groupby("occupation")["vacancies"].sum().sort_values(ascending=False).reset_index().head(5)

occ_list = [top_occupations["occupation"][0]]
print(occ_list[0])

# df_mart["workplace_region"]
df_mart
top_municipality = df_mart.groupby("workplace_municipality")["vacancies"].sum().sort_values(ascending=False).reset_index().head(5)
top_municipality


Vårdbiträde


,workplace_municipality,vacancies
0,Göteborg,24878
1,Malmö,6906
2,Linköping,3641
3,Stockholm,3368
4,Örebro,2854


In [55]:
import plotly.express as px
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
# vilket yrke har högst andel lediga jobb? linjediagram med en linje per yrke
df_technical_occ = vacancies(df_technical, "occupation")
df_education_occ = vacancies(df_education, "occupation")
df_healthcare_occ = vacancies(df_healthcare, "occupation")
df_mart_occ = vacancies(df_mart, "occupation")


df_technical_occ
occupation_chart1 = px.bar(df_technical_occ.head(5), x="occupation", y="total vacancies")
occupation_chart2 = px.bar(df_education_occ.head(5), x="occupation", y="total vacancies")
occupation_chart3 = px.bar(df_healthcare_occ.head(5), x="occupation", y="total vacancies")

subfig = make_subplots(rows=1, cols=3, subplot_titles=("Technical","Education", "Healthcare"))

for trace in occupation_chart1.data: 
    subfig.add_trace(trace, row=1, col=1)
for trace in occupation_chart2.data: 
    subfig.add_trace(trace, row=1, col=2)
for trace in occupation_chart3.data: 
    subfig.add_trace(trace, row=1, col=3)

subfig.update_layout(title_text="Topp 5 yrken med flest lediga jobb - per yrkesgrupp",title_x=0.5, width=1200, title_font=dict(size=20))
subfig.update_yaxes(title_text="Totalt antal lediga jobb", row=1, col=1, title_font=dict(size=14) )
subfig.update_xaxes()


subfig.show()

In [56]:
# vilken stad har högst andel lediga jobb? bardiagram med en linje per yrkesgrupp
# y = total vacancies
# x = region
# gör subplottar för varje! 
import plotly.graph_objects as go
from plotly.subplots import make_subplots

df_technical_region = vacancies(df_technical, "workplace_region")
df_education_region = vacancies(df_education, "workplace_region")
df_healthcare_region = vacancies(df_healthcare, "workplace_region")

region_chart1 = px.bar(df_education_region.head(5), y="total vacancies", x="workplace_region")
region_chart2 = px.bar(df_healthcare_region.head(5), y="total vacancies", x="workplace_region")
region_chart3 = px.bar(df_technical_region.head(5), y="total vacancies", x="workplace_region")


fig = make_subplots(rows=1, cols=3, subplot_titles=("Education", "Healthcare","Technical"))
for trace in region_chart1.data: 
    fig.add_trace(trace, row=1, col=1)
for trace in region_chart2.data: 
    fig.add_trace(trace, row=1, col=2)
for trace in region_chart3.data: 
    fig.add_trace(trace, row=1, col=3)

fig.update_layout(title_text="Topp 5 regioner med flest lediga jobb per yrkesgrupp", title_x=0.5, width=1200, title_font=dict(size=20))   
fig.update_yaxes(title_text="Total vacancies", col=1, title_font=dict(size=14)) 
fig.show()

In [57]:
# 
## Sålla per yrkesgrupp (occ_field eller df:arna): 
# px.bar(df_mart, x="workplace_region", y="vacancies", color="occupation_field", barmode="group")
df_mart[df_mart["occupation_field"] == "Pedagogik"].groupby("occupation")["vacancies"].sum().sort_values(ascending=False).reset_index()


,occupation,vacancies
0,Elevassistent,4879
1,"Ämneslärare, 7-9",4489
2,"Grundlärare, 4-6",3878
3,"Grundlärare, fritidshem",2299
4,Lärarassistent,1516
5,Speciallärare,1464
6,Lärare i gymnasieskolan,1229
7,"Grundlärare, förskoleklass och 1-3",1208
8,Förskollärare,916
9,Specialpedagog,565


In [58]:
# vilken arbetsgivare har högst andel lediga jobb? barchart
top_sweden_employers = df_mart.groupby("employer_name")["vacancies"].sum().sort_values(ascending=False).reset_index(name="total vacancies")
px.bar(top_sweden_employers.head(10), y="total vacancies", x= "employer_name")

In [95]:
# vilken yrkesgrupp har högst andel lediga jobb? occ_group      barchart
top_occupation_groups = df_mart.groupby("occupation")["vacancies"].sum().sort_values(ascending=False).reset_index(name="total vacancies")
px.bar(top_occupation_groups.head(10), y="total vacancies", x="occupation")

In [89]:
# andel jobb som kräver körkort + egen bil + erfarenhet
df_mart.columns
df_attributes = df_mart.groupby("occupation_field")[["experience_required", "driver_licence", "access_to_own_car"]].sum().reset_index()
df_attributes
# px.bar(df_attributes, y=["experience_required", "driver_licence", "access_to_own_car"], x="occupation_field")

# px.pie(df_attributes, values="count", names=["experience_required", "driver_licence", "access_to_own_car"])
df_attributes = df_mart.groupby("occupation_field")[["experience_required", "driver_licence", "access_to_own_car"]].sum().reset_index()
experience = df_attributes["experience_required"]
licence = df_attributes["driver_licence"]
car = df_attributes["access_to_own_car"]
experience
df_attributes
# df_attributes[df_attributes["occupation_field"] == "Pedagogik"]

,occupation_field,experience_required,driver_licence,access_to_own_car
0,Hälso- och sjukvård,5180,1004,18
1,Pedagogik,7909,437,7
2,Yrken med teknisk inriktning,5551,710,32


In [ ]:
# pie-chart! 
# mart_full_time = df_mart[df_mart["scope_of_work_max"] == 100].groupby("occupation_field")["vacancies"].sum().sort_values(ascending=False).reset_index(name="total vacancies")
mart_full_time = df_mart.groupby("occupation_field")["vacancies"].sum().sort_values(ascending=False).reset_index(name="total vacancies")
mart_full_time
px.pie(mart_full_time,values="total vacancies",names="occupation_field",title="Andel jobb i sverige")


In [ ]:
mart_full_time = df_mart.groupby("occupation_field")["vacancies"].sum().sort_values(ascending=False).reset_index(name="total vacancies")
mart_full_time

,occupation_field,total vacancies
0,Hälso- och sjukvård,33899
1,Pedagogik,26480
2,Yrken med teknisk inriktning,9961


In [ ]:

# avancerat: Vilken stad har störst variation av yrken? 
df_mart

,headline,occupation,occupation_group,occupation_field,description,duration,working_hours_type,scope_of_work_min,scope_of_work_max,vacancies,employer_name,workplace_region,workplace_municipality,experience_required,driver_licence,access_to_own_car
0,Diagnostik Expert på Amaris Consulting,Fordonsingenjör,Ingenjörer och tekniker inom maskinteknik,Yrken med teknisk inriktning,Är du en expert inom diagnostik och söker nya ...,Tills vidare,Heltid,100,100,1,Amaris AB,Västra Götalands län,Göteborg,True,False,False
1,Junior Kvalitets- och Miljöingenjör till JOAB,"Kvalitetsingenjör, maskin/Kvalitetstekniker, m...",Ingenjörer och tekniker inom maskinteknik,Yrken med teknisk inriktning,Om företaget\nJOAB är en ledande aktör inom på...,Tills vidare,Heltid,100,100,1,Skill Rekrytering & Bemanning AB,Västra Götalands län,Göteborg,False,False,False
2,Konstruktionschef till Scanditronix Magnet AB,"Konstruktör, tillverkningsindustri",Ingenjörer och tekniker inom maskinteknik,Yrken med teknisk inriktning,Beskrivning\nVi söker en erfaren maskinkonstru...,Tills vidare,Heltid,100,100,1,Jobcenter in Sweden AB,Kronobergs län,Växjö,True,True,True
3,Konstruktionschef till Scanditronix Magnet AB,"Konstruktör, tillverkningsindustri",Ingenjörer och tekniker inom maskinteknik,Yrken med teknisk inriktning,Beskrivning\nVi söker en erfaren maskinkonstru...,Tills vidare,Heltid,100,100,1,Jobcenter in Sweden AB,Kronobergs län,Ljungby,True,True,True
4,Governance and Information Security Officer,"Civilingenjör, teknisk fysik",Övriga civilingenjörsyrken,Yrken med teknisk inriktning,"Want to influence the bigger picture? At Saab,...",Tills vidare,Heltid,100,100,1,SAAB AB,Östergötlands län,Linköping,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33427,Lärare årskurs 1-3 Landamäreskolan,"Grundlärare, förskoleklass och 1-3",Grundskollärare,Pedagogik,I grundskoleförvaltningen har vi en trygg och ...,6 månader eller längre,Heltid,100,100,1,GÖTEBORGS KOMMUN,Västra Götalands län,Göteborg,False,False,False
33428,Lärare åk 4 till Rosendalsskolan (vikariat läs...,"Grundlärare, 4-6",Grundskollärare,Pedagogik,I grundskoleförvaltningen har vi en trygg och ...,6 månader eller längre,Heltid,100,100,1,GÖTEBORGS KOMMUN,Västra Götalands län,Göteborg,False,False,False
33429,Lärare i slöjd till skolområde Sydväst 20,"Grundlärare, 4-6",Grundskollärare,Pedagogik,I grundskoleförvaltningen har vi en trygg och ...,Tills vidare,Heltid,100,100,1,GÖTEBORGS KOMMUN,Västra Götalands län,Göteborg,False,False,False
33430,Lärare i slöjd till skolområde Sydväst 20 (vik...,"Grundlärare, 4-6",Grundskollärare,Pedagogik,I grundskoleförvaltningen har vi en trygg och ...,6 månader eller längre,Heltid,100,100,1,GÖTEBORGS KOMMUN,Västra Götalands län,Göteborg,False,False,False
